In [37]:
import os
os.chdir(r"c:\Users\rohan\Documents\Flutter Projects\saas-subscription-analytics")
os.getcwd()

'c:\\Users\\rohan\\Documents\\Flutter Projects\\saas-subscription-analytics'

In [38]:
import pandas as pd

accounts = pd.read_csv("data/processed/accounts_cleaned.csv")
subs = pd.read_csv("data/processed/subscriptions_cleaned.csv")
churn = pd.read_csv("data/processed/churn_events_cleaned.csv")
usage = pd.read_csv("data/processed/feature_usage_cleaned.csv")
tickets = pd.read_csv("data/processed/support_tickets_cleaned.csv")

In [39]:
date_cols_subs = ["start_date", "end_date"]
for c in date_cols_subs:
    subs[c] = pd.to_datetime(subs[c])

churn["churn_date"] = pd.to_datetime(churn["churn_date"])
usage["usage_date"] = pd.to_datetime(usage["usage_date"])
tickets["submitted_at"] = pd.to_datetime(tickets["submitted_at"])
tickets["closed_at"] = pd.to_datetime(tickets["closed_at"])
accounts["signup_date"] = pd.to_datetime(accounts["signup_date"])

In [40]:
active_subs = subs[subs["end_date"].isna()]

active_customers = active_subs["account_id"].nunique()
active_customers

500

In [41]:
mrr = active_subs["mrr_amount"].sum()
mrr

np.int64(10159608)

In [42]:
subs["start_month"] = subs["start_date"].dt.to_period("M")
mrr_trend = subs.groupby("start_month")["mrr_amount"].sum()
mrr_trend

start_month
2023-01       4684
2023-02      11079
2023-03      25885
2023-04      41788
2023-05      85919
2023-06      74987
2023-07     120422
2023-08     165621
2023-09     116222
2023-10     182518
2023-11     217219
2023-12     267264
2024-01     276933
2024-02     365442
2024-03     421916
2024-04     445858
2024-05     632549
2024-06     537758
2024-07     707816
2024-08     648426
2024-09     992366
2024-10    1173608
2024-11    1549040
2024-12    2273427
Freq: M, Name: mrr_amount, dtype: int64

In [43]:
arr = mrr * 12
arr


np.int64(121915296)

In [44]:
churn["month"] = churn["churn_date"].dt.to_period("M")

monthly_churn = churn.groupby("month")["account_id"].nunique()
monthly_churn


month
2023-01     1
2023-03     5
2023-04     3
2023-05     3
2023-06     5
2023-07     6
2023-08     6
2023-09     6
2023-10    10
2023-11    11
2023-12    16
2024-01    18
2024-02    10
2024-03    24
2024-04    23
2024-05    25
2024-06    38
2024-07    33
2024-08    40
2024-09    48
2024-10    60
2024-11    60
2024-12    96
Freq: M, Name: account_id, dtype: int64

In [45]:
starting_customers = accounts["account_id"].nunique()
total_churned = churn["account_id"].nunique()

churn_rate = total_churned / starting_customers
churn_rate


0.704

In [46]:
expansion = subs[subs["upgrade_flag"] == True]["mrr_amount"].sum()
contraction = subs[subs["downgrade_flag"] == True]["mrr_amount"].sum()
expansion, contraction

(np.int64(1262997), np.int64(459366))

In [47]:
churned_subs = subs[subs["churn_flag"] == True]
churned_mrr = churned_subs["mrr_amount"].sum()

starting_mrr = subs["mrr_amount"].sum()

nrr = (starting_mrr + expansion - contraction - churned_mrr) / starting_mrr
nrr


np.float64(0.9668827605025494)

In [48]:
upgrade_rate = subs["upgrade_flag"].mean()
upgrade_rate


np.float64(0.1058)

In [49]:
tickets["resolution_time_hours"].mean()


np.float64(35.861)

In [50]:
tickets["satisfaction_score"].mean()


np.float64(3.981276595744681)

In [51]:
tickets.groupby("account_id")["ticket_id"].count().mean()


np.float64(4.065040650406504)

In [52]:
usage_daily = usage.groupby("usage_date")["usage_count"].sum()
usage_daily.head()


usage_date
2023-01-01    321
2023-01-02    272
2023-01-03    418
2023-01-04    392
2023-01-05    296
Name: usage_count, dtype: int64

In [53]:
usage.groupby("feature_name")["usage_count"].sum().sort_values(ascending=False).head()


feature_name
feature_32    6686
feature_15    6621
feature_6     6546
feature_20    6541
feature_34    6536
Name: usage_count, dtype: int64

In [ ]:
usage.groupby("subscription_id")["usage_count"].sum().describe()


count    4967.000000
mean       50.437890
std        23.116423
min         3.000000
25%        33.000000
50%        48.000000
75%        66.000000
max       148.000000
Name: usage_count, dtype: float64

: 